In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
df = pd.read_csv("townlands-no-geom.csv")

/home/ismael/.local/share/virtualenvs/JupyterLab-lsuK08Tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
df.head()

,OSM_ID,NAME_TAG,NAME_GA,NAME_EN,ALT_NAME,ALT_NAME_G,OSM_USER,OSM_TIMEST,ATTRIBUTIO,LOGAINM_RE,CO_NAME,CO_OSM_ID,CO_LOGAINM,CP_NAME,CP_OSM_ID,CP_LOGAINM,ED_NAME,ED_OSM_ID,ED_LOGAINM,BAR_NAME,BAR_OSM_ID,BAR_LOGAIN,T_IE_URL,AREA,LATITUDE,LONGITUDE,EPOCH_TSTM
0,-4726163,Gortgower,NaN,Gortgower,NaN,NaN,MikeOS,2015-03-24 19:44:26+01,NaN,22431,Kerry,-332622.0,100005.0,Valencia,-6102759.0,1128,Valencia,-6867907.0,NaN,Iveragh,-5304835.0,88,http://www.townlands.ie/kerry/iveragh/valencia...,2.309681e+05,51.924771,-10.330612,1.427223e+09
1,-6419252,Long Island,NaN,Long Island,NaN,NaN,Boggedy,2016-07-17 15:35:17+02,NaN,22386,Kerry,-332622.0,100005.0,Killemlagh,-6280381.0,1095,NaN,NaN,NaN,Iveragh,-5304835.0,88,http://www.townlands.ie/kerry/iveragh/killemla...,4.133401e+04,51.879500,-10.403492,1.468763e+09
2,-6205326,Ballybrack,An Baile Breac,NaN,NaN,NaN,Stephen_Co_Antrim,2016-05-11 22:35:00+02,NaN,NaN,Kerry,-332622.0,100005.0,Kilquane,-6267057.0,1104,Kilquann,-6342994.0,NaN,Corkaguiny,-5304974.0,91,http://www.townlands.ie/kerry/corkaguiny/kilqu...,4.968431e+06,52.218589,-10.283522,1.462999e+09
3,-6196255,Castlequarter,CeathrÃº an ChaisleÃ¡in,NaN,NaN,NaN,Stephen_Co_Antrim,2016-05-07 18:36:24+02,NaN,NaN,Kerry,-332622.0,100005.0,Marhin,-6266901.0,1113,Marhin,-6341710.0,NaN,Corkaguiny,-5304974.0,91,http://www.townlands.ie/kerry/corkaguiny/marhi...,6.981506e+05,52.169369,-10.393195,1.462639e+09
4,-6301754,Granshagh,An GhrÃ¡inseach,Granshagh,NaN,NaN,Teester,2016-06-10 01:59:34+02,NaN,23810,Kerry,-332622.0,100005.0,Kilcaragh,-6377019.0,1078,Ballincloher,-6855337.0,NaN,Clanmaurice,-5305317.0,94,http://www.townlands.ie/kerry/clanmaurice/kilc...,8.282486e+05,52.382347,-9.674393,1.465517e+09


In [5]:
neighborhoods = df[df.CO_NAME == "Dublin"].reset_index(drop=True)
neighborhoods = neighborhoods[['CP_NAME','LATITUDE','LONGITUDE']]
neighborhoods = neighborhoods.groupby(['CP_NAME']).mean()
neighborhoods.reset_index(level=0, inplace=True)
neighborhoods.rename(columns={"CP_NAME": "Neighborhood", "LATITUDE": "Latitude", "LONGITUDE": "Longitude"}, inplace=True)
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Aderrig,53.340213,-6.474632
1,Artaine,53.384701,-6.217102
2,Baldongan,53.553805,-6.126480
3,Baldoyle,53.403104,-6.139341
4,Balgriffin,53.412817,-6.183087


In [6]:
neighborhoods.to_csv("dublin_neighborhoods.csv", index=False)

In [7]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
address = 'Dublin, Ireland'

geolocator = Nominatim(user_agent="dublin_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dublin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dublin are 53.3497645, -6.2602732.


In [9]:
# create map of Dublin using latitude and longitude values
map_dublin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dublin)  
    
map_dublin

In [ ]:
with open('pass.txt','r') as f:
    output = f.read()
CLIENT_ID,CLIENT_SECRET,VERSION,BIN = output.split(',')

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500, LIMIT=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
dublin_venues = getNearbyVenues(names=neighborhoods["Neighborhood"],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Aderrig
Artaine
Baldongan
Baldoyle
Balgriffin
Ballyboghil
Ballyfermot
Ballymadun
Balrothery
Balscaddan
Booterstown
Castleknock
Chapelizod
Cloghran
Clondalkin
Clonmethan
Clonsilla
Clontarf
Clonturk
Coolock
Cruagh
Crumlin
Dalkey
Donabate
Donnybrook
Drimnagh
Esker
Finglas
Garristown
Glasnevin
Grallagh
Grangegorman
Hollywood
Holmpatrick
Howth
Kilbarrack
Kilbride
Kilgobbin
Kill
Killeek
Killester
Killiney
Killossery
Kilmactalway
Kilmacud
Kilmahuddrick
Kilsallaghan
Kiltiernan
Kinsaley
Leixlip
Lucan
Lusk
Malahide
Monkstown
Mulhuddart
Naul
Newcastle
Old Connaught
Palmerston
Palmerstown
Portmarnock
Portraine
Raheny
Rathcoole
Rathfarnham
Rathmichael
Saggart
Santry
St Mark's
St. Catherine's
St. George's
St. James'
St. Jude's
St. Margaret's
St. Nicholas Without
St. Peter's
Stillorgan
Swords
Tallaght
Taney
Tully
Ward
Westpalstown
Whitechurch


In [13]:
print(dublin_venues.shape)
dublin_venues.head()

(2173, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aderrig,53.340213,-6.474632,Finnstown Castle Hotel,53.339602,-6.460888,Hotel
1,Aderrig,53.340213,-6.474632,Adamstown Railway Station,53.337099,-6.462584,Train Station
2,Aderrig,53.340213,-6.474632,Peacock Restaurant,53.339547,-6.461501,Gastropub
3,Aderrig,53.340213,-6.474632,Londis,53.336752,-6.457966,Convenience Store
4,Aderrig,53.340213,-6.474632,PDM Construction,53.350451,-6.469920,Building


In [14]:
dublin_venues.to_csv("dublin_venues.csv", index=False)